# Pretrain Policy with  `MPC dataset`

In [1]:
import numpy as np
import pickle
from typing import Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal
from mujoco_parser import MuJoCoParserClass
import torch.optim as optim

from tqdm import tqdm
import matplotlib.pyplot as plt
from util import r2quat

from torch.utils.data import DataLoader
from torch.utils.data import Dataset

from policy import GaussianPolicy

import mediapy as media

### Load  `MPC dataset`

In [2]:
# with open(file='../data/MPC_dataset_240315_ctrl_minimize.pkl', mode='rb') as f:
#     dataset = pickle.load(f)

# horizon = 50
# action = torch.Tensor(dataset['action'])
# root_pos = torch.Tensor(dataset['root_pos'])
# root_rot = torch.Tensor(dataset['root_rot'])
# root_vel = torch.Tensor(dataset['root_vel'])
# root_ang_vel = torch.Tensor(dataset['root_ang_vel'])
# dof_pos = torch.Tensor(dataset['dof_pos'])
# dof_vel = torch.Tensor(dataset['dof_vel'])
# local_key_pos = torch.Tensor(dataset['local_key_pos'])

# del dataset

# with open(file='../data/MPC_dataset_240319_noise.pkl', mode='rb') as f:
#     dataset = pickle.load(f)

# horizon = 50
# action = torch.tensor(np.concatenate((action, dataset['action'])), dtype=torch.float32)
# root_pos = torch.tensor(np.concatenate((root_pos, dataset['root_pos'])), dtype=torch.float32)
# root_rot = torch.tensor(np.concatenate((root_rot, dataset['root_rot'])), dtype=torch.float32)
# root_vel = torch.tensor(np.concatenate((root_vel, dataset['root_vel'])), dtype=torch.float32)
# root_ang_vel = torch.tensor(np.concatenate((root_ang_vel, dataset['root_ang_vel'])), dtype=torch.float32)
# dof_pos = torch.tensor(np.concatenate((dof_pos, dataset['dof_pos'])), dtype=torch.float32)
# dof_vel = torch.tensor(np.concatenate((dof_vel, dataset['dof_vel'])), dtype=torch.float32)
# local_key_pos = torch.tensor(np.concatenate((local_key_pos, dataset['local_key_pos'])), dtype=torch.float32)

# del dataset

# with open(file='../data/MPC_dataset_240321_bigger_noise.pkl', mode='rb') as f:
#     dataset = pickle.load(f)

# horizon = 50
# action = torch.tensor(np.concatenate((action, dataset['action'])), dtype=torch.float32)
# root_pos = torch.tensor(np.concatenate((root_pos, dataset['root_pos'])), dtype=torch.float32)
# root_rot = torch.tensor(np.concatenate((root_rot, dataset['root_rot'])), dtype=torch.float32)
# root_vel = torch.tensor(np.concatenate((root_vel, dataset['root_vel'])), dtype=torch.float32)
# root_ang_vel = torch.tensor(np.concatenate((root_ang_vel, dataset['root_ang_vel'])), dtype=torch.float32)
# dof_pos = torch.tensor(np.concatenate((dof_pos, dataset['dof_pos'])), dtype=torch.float32)
# dof_vel = torch.tensor(np.concatenate((dof_vel, dataset['dof_vel'])), dtype=torch.float32)
# local_key_pos = torch.tensor(np.concatenate((local_key_pos, dataset['local_key_pos'])), dtype=torch.float32)

# del dataset

# with open(file='../data/MPC_dataset_240323_biggger_noise.pkl', mode='rb') as f:
#     dataset = pickle.load(f)

# horizon = 50
# action = torch.tensor(np.concatenate((action, dataset['action'])), dtype=torch.float32)
# root_pos = torch.tensor(np.concatenate((root_pos, dataset['root_pos'])), dtype=torch.float32)
# root_rot = torch.tensor(np.concatenate((root_rot, dataset['root_rot'])), dtype=torch.float32)
# root_vel = torch.tensor(np.concatenate((root_vel, dataset['root_vel'])), dtype=torch.float32)
# root_ang_vel = torch.tensor(np.concatenate((root_ang_vel, dataset['root_ang_vel'])), dtype=torch.float32)
# dof_pos = torch.tensor(np.concatenate((dof_pos, dataset['dof_pos'])), dtype=torch.float32)
# dof_vel = torch.tensor(np.concatenate((dof_vel, dataset['dof_vel'])), dtype=torch.float32)
# local_key_pos = torch.tensor(np.concatenate((local_key_pos, dataset['local_key_pos'])), dtype=torch.float32)

# del dataset

# with open(file='../data/MPC_dataset_240324_bigggger_noise.pkl', mode='rb') as f:
#     dataset = pickle.load(f)

# horizon = 50
# action = torch.tensor(np.concatenate((action, dataset['action'])), dtype=torch.float32)
# root_pos = torch.tensor(np.concatenate((root_pos, dataset['root_pos'])), dtype=torch.float32)
# root_rot = torch.tensor(np.concatenate((root_rot, dataset['root_rot'])), dtype=torch.float32)
# root_vel = torch.tensor(np.concatenate((root_vel, dataset['root_vel'])), dtype=torch.float32)
# root_ang_vel = torch.tensor(np.concatenate((root_ang_vel, dataset['root_ang_vel'])), dtype=torch.float32)
# dof_pos = torch.tensor(np.concatenate((dof_pos, dataset['dof_pos'])), dtype=torch.float32)
# dof_vel = torch.tensor(np.concatenate((dof_vel, dataset['dof_vel'])), dtype=torch.float32)
# local_key_pos = torch.tensor(np.concatenate((local_key_pos, dataset['local_key_pos'])), dtype=torch.float32)

# del dataset

# with open(file='../data/MPC_dataset_240324_bigggger_noise-2.pkl', mode='rb') as f:
#     dataset = pickle.load(f)

# horizon = 50
# action = torch.Tensor(dataset['action'])
# root_pos = torch.Tensor(dataset['root_pos'])
# root_rot = torch.Tensor(dataset['root_rot'])
# root_vel = torch.Tensor(dataset['root_vel'])
# root_ang_vel = torch.Tensor(dataset['root_ang_vel'])
# dof_pos = torch.Tensor(dataset['dof_pos'])
# dof_vel = torch.Tensor(dataset['dof_vel'])
# local_key_pos = torch.Tensor(dataset['local_key_pos'])

# del dataset

with open(file='../data/SMPL_MPC_dataset_240424.pkl', mode='rb') as f:
    dataset = pickle.load(f)

horizon = 200
action = torch.Tensor(dataset['action'])
root_pos = torch.Tensor(dataset['root_pos'])
root_rot = torch.Tensor(dataset['root_rot'])
root_vel = torch.Tensor(dataset['root_vel'])
root_ang_vel = torch.Tensor(dataset['root_ang_vel'])
dof_pos = torch.Tensor(dataset['dof_pos'])
dof_vel = torch.Tensor(dataset['dof_vel'])
local_key_pos = torch.Tensor(dataset['local_key_pos'])

del dataset

# qpos_batch = qpos_batch[:, :]
# qpos_batch = torch.cat((qpos_batch[:-2, :],qpos_batch[1:-1, :],qpos_batch[2:, :]),dim=1)
# qvel_batch = torch.cat((qvel_batch[:-2, :],qvel_batch[1:-1, :],qvel_batch[2:, :]),dim=1)

obs = torch.cat((root_pos[:,2].reshape(-1, 1), root_rot, root_vel, root_ang_vel, dof_pos, dof_vel, local_key_pos.reshape(-1, 12)), dim=1)
# obs = torch.cat((root_pos, root_rot, root_vel, root_ang_vel, dof_pos, dof_vel), dim=1)

# action_batch = action_batch[2:]
obs_dim = obs.shape[1]
action_dim = action.shape[1]
hidden_dim = 256

print("obs : ", obs.shape)
print("action : ", action.shape)


obs :  torch.Size([40000, 97])
action :  torch.Size([40000, 37])


In [3]:

xml_path = '../asset/smpl_rig/scene_smpl_rig.xml'
env = MuJoCoParserClass(name='smplrig',rel_xml_path=xml_path,VERBOSE=True)
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs:
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]
print ("Done.")

dt:[0.0083] HZ:[120]
n_dof (=nv):[43]
n_geom:[67]
geom_names:['floor', 'base', 'base2lpelvis', 'base2rpelvis', 'base2spine1', 'spine2spine', 'spine2spine2', 'spine2spine3', 'spine2lcollar', 'spine2rcollar', 'neck2head', 'head', 'nose', 'lcollar2lshoulder', 'lshoulder2lelbow', 'lelbow2lwrist', 'lwrist2lindex1', 'lwrist2lmiddle1', 'lwrist2lpinky1', 'lwrist2lring1', 'lwrist2lthumb1', 'lindex1-lindex2', 'lindex2-lindex3', 'lindex3-lindextip', 'lmiddle1-lmiddle2', 'lmiddle2-lmiddle3', 'lmiddl3-lmiddletip', 'lring1-lring2', 'lring2-lring3', 'lring3-lringtip', 'lpinky1-lpinky2', 'lpinky2-lpinky3', 'lpinky3-lpinkytip', 'lthumb1-lthumb2', 'lthumb2-lthumb3', 'lthumb3-lthumbtip', 'rcollar2rshoulder', 'rshoulder2relbow', 'relbow2rwrist', 'rwrist2rindex1', 'rwrist2rmiddle1', 'rwrist2rpinky1', 'rwrist2rring1', 'rwrist2rthumb1', 'rindex1-rindex2', 'rindex2-rindex3', 'rindex3-rindextip', 'rmiddle1-rmiddle2', 'rmiddle2-rmiddle3', 'rmiddle3-rmiddletip', 'rring1-rring2', 'rring2-rring3', 'rring3-rringtip

### Load reference walking motion

In [4]:
motion_name = 'smplrig_walk_optimized_recon'
pkl_path = '../data/%s.pkl'%(motion_name)
with open(pkl_path, 'rb') as f:
    data_reference = pickle.load(f)
print ("[%s] loaded."%(pkl_path))

[../data/smplrig_walk_optimized_recon.pkl] loaded.


In [5]:
KEY_BODY_NAMES = ["right_ankle", "left_ankle", "right_wrist", "left_wrist"]
key_body_ids= np.array(env.get_idxs_body(KEY_BODY_NAMES))

In [6]:
root_pos = torch.Tensor(data_reference['p_root'])
root_rot = torch.Tensor(data_reference['R_root'])
root_vel = torch.Tensor(data_reference['v_root'])
root_ang_vel = torch.Tensor(data_reference['w_root'])
dof_pos = torch.Tensor(data_reference['qpos'])
dof_vel = torch.Tensor(data_reference['qvel'])
local_key_pos = torch.Tensor((data_reference['xpos'][:,key_body_ids,:]-data_reference['xpos'][:,1,:].reshape(315,1,3)).reshape(315,12))

del data_reference


# qpos_batch = qpos_batch[:, :]
# qpos_batch = torch.cat((qpos_batch[:-2, :],qpos_batch[1:-1, :],qpos_batch[2:, :]),dim=1)
# qvel_batch = torch.cat((qvel_batch[:-2, :],qvel_batch[1:-1, :],qvel_batch[2:, :]),dim=1)

obs_ref = torch.cat((root_pos[:,2].reshape(-1, 1), root_rot, root_vel, root_ang_vel, dof_pos, dof_vel, local_key_pos.reshape(-1, 12)), dim=1)


In [7]:
policy = GaussianPolicy(
    input_dim=obs_dim*2,
    output_dim=action_dim,
    hidden_dim=hidden_dim,
    is_deterministic=False,
)

RunningMeanStd:  194
RunningMeanStd:  37


In [8]:
# class MPCDataset(Dataset):
#     def __init__(self, obs, act):
#         self.obs = obs
#         # self.obs = (self.obs - self.obs.mean(dim=0)) / self.obs.std(dim=0)
#         self.act = act
#         # self.act = (self.act - self.act.mean(dim=0)) / self.act.std(dim=0)
#         assert self.obs.shape[0] == self.act.shape[0]

#     def __len__(self):
#         return self.obs.shape[0]

#     def __getitem__(self,idx):
#         return self.obs[idx], self.act[idx]

In [9]:
class MPCDataset(Dataset):
    def __init__(self, obs, act, obs_ref, horizon=50):
        # self.obs = obs
        # self.obs = (self.obs - self.obs.mean(dim=0)) / self.obs.std(dim=0)
        self.act = act[:]# * env.model.actuator_gear[:, 0]
        self.obs_ref = obs_ref
        # self.act = (self.act - self.act.mean(dim=0)) / self.act.std(dim=0)
        # self.next_obs = obs[1:]
        self.obs = obs[:]
        self.horizon = horizon

        assert self.obs.shape[0] == self.act.shape[0]

    def __len__(self):
        return self.obs.shape[0]

    def __getitem__(self,idx):
        first_frame = int(idx/self.horizon)
        t=min(first_frame+idx-first_frame*self.horizon+1,314)
        # if idx % self.horizon == 49:
        #     idx-=1
        return torch.cat((self.obs[idx], self.obs_ref[t])), self.act[idx]

In [10]:
train_dataset = MPCDataset(obs, action, obs_ref, horizon)
train_dataloader = DataLoader(train_dataset, batch_size=2048, shuffle=True)
# test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [12]:
num_epoch = 100
optimizer = optim.Adam(policy.parameters(), lr=1e-5, betas=(0.9, 0.95), weight_decay=0.01)
loss = torch.nn.MSELoss()
def criterion(output: torch.tensor, y: torch.tensor, mean, std, log_prob):
    diff = output - y

    normal = Normal(mean, std)
    GT_neglogp = -normal.log_prob(y)

    logp_error = torch.mean(GT_neglogp)
    logp_coeff = 1e-3
    pred_error = (torch.mean(torch.abs(diff)))

    return pred_error + logp_error * logp_coeff
    # return torch.mean(torch.sqrt(torch.mean(torch.square(diff),dim=1)))
    # return loss(output, y)

policy.train()
policy.running_mean_std.train()
# policy.action_mean_std.train()
for epoch in range(num_epoch):

    with tqdm(train_dataloader, unit="batch") as tepoch:
        
        for x, y in tepoch:
            
            tepoch.set_description(f"Epoch {epoch+1}")

            optimizer.zero_grad()
                
            pred, mean, std, log_prob = policy(x) # supervised
            l = criterion(pred, y, mean, std, log_prob)
            l.backward()
            optimizer.step()
            
            tepoch.set_postfix(loss=l.item())


Epoch 100: 100%|██████████| 20/20 [00:00<00:00, 25.09batch/s, loss=0.0492]


### Test pretrain model

In [13]:
PLOT = False
policy.eval()
policy.running_mean_std.eval()
policy.is_deterministic = True

In [14]:
# Set which joints to control
ctrl_joint_names = env.ctrl_joint_names # <==
joint_idxs_fwd = env.get_idxs_fwd(joint_names=ctrl_joint_names)
joint_idxs_jac = env.get_idxs_jac(joint_names=ctrl_joint_names)
q_ctrl_init = env.get_qpos_joints(ctrl_joint_names)
n_ctrl_joint = len(ctrl_joint_names)

In [15]:
env.data.xpos[1]

array([0.  , 0.  , 0.95])

In [16]:
# Initialize MuJoCo viewer
env.init_viewer(viewer_title='SMPL',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=False)
env.update_viewer(azimuth=152,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
env.reset()

image_list = []
for t in np.arange(30,315):
    rp = root_pos[t]
    rr = root_rot[t]
    rv = root_vel[t]
    rw = root_ang_vel[t]
    qpos = dof_pos[t]
    qvel = dof_vel[t]
#     print(rr[0])
    env.data.qvel = torch.cat((rv, rw, qvel))
    env.forward(torch.cat((rp, rr, qpos)))
    for h in range(100):
        ctrl = action[t+h]
        # state = obs[50*t+h]
        state = torch.from_numpy(np.concatenate((env.get_p_body('base')[2:], r2quat(env.get_R_body('base')), env.get_qvel_joint('base')[0:3], env.get_qvel_joint('base')[3:6], env.data.qpos[env.rev_joint_idxs+6].copy(), env.data.qvel[env.rev_joint_idxs+5].copy(), (env.data.xpos[key_body_ids]-env.get_p_body('base')).reshape(12)), axis=-1)).type(torch.float32)
        # next_state = torch.from_numpy(np.concatenate((root_pos[horizon*t+h+1,2:],root_rot[horizon*t+h+1],root_vel[horizon*t+h+1],root_ang_vel[horizon*t+h+1],dof_pos[horizon*t+h+1],dof_vel[horizon*t+h+1],local_key_pos[horizon*t+h+1].reshape(12)), axis=-1))
        next_idx = min(t+h+1,314)
        next_state = torch.from_numpy(np.concatenate((root_pos[next_idx,2:],root_rot[next_idx],root_vel[next_idx],root_ang_vel[next_idx],dof_pos[next_idx],dof_vel[next_idx],local_key_pos[next_idx]), axis=-1))
        ext_state = torch.cat((state, next_state))
        
        pred_ctrl, _, _, _ = policy(ext_state)
        pred_ctrl = pred_ctrl.detach().numpy()
        # pred_ctrl=policy.action_mean_std(pred_ctrl, unnorm=True).detach().numpy() / env.model.actuator_gear[:,0]

        env.step(pred_ctrl)
        if env.loop_every(tick_every=1):
            # Plot world frame
            env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                    PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
            env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                    PLOT_AXIS=False,label="tick:[%d]"%(env.tick))

            # # Stand
            # for p_GT in GT['xpos'][int(t+h)]:
            #     env.plot_sphere(p=p_GT,r=0.02,rgba=[1,0.2,0.2,1])
            
            env.render()

            # if SAVE_VID:
            #     # Save iamges
            #     png_path = '../temp/smplrig_MPC_%05d.png'%(env.tick)
            #     create_folder_if_not_exists(png_path)
            #     image = cv2.cvtColor(env.grab_image(),cv2.COLOR_RGB2BGR)
            #     cv2.imwrite(png_path,image)
            if PLOT:
                fig, ax = plt.subplots()
                plt.ylim(-1, 1)
                ax.bar(np.arange(action_dim)-0.2, ctrl, width = 0.35, align='edge', label='GT')
                ax.bar(np.arange(action_dim)+0.2, pred_ctrl, width = 0.35, align='edge', label='Predicted')
                ax.set_title("loss: {}".format(
                    torch.mean(torch.abs(ctrl-pred_ctrl))
                ))
                ax.legend()
                ax.margins(0)
                fig.canvas.draw()

                image_from_plot = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
                image_from_plot = image_from_plot.reshape(fig.canvas.get_width_height()[::-1] + (3,))

                image_list.append(image_from_plot)


2024-04-25 00:38:51.718 python[34347:7393931] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


Pressed ESC
Quitting.


Exception: GLFW window does not exist but you tried to render.

In [ ]:
LOAD = False
SAVE = False

file_name = "240412_smpl_walk_obs_g.pth"
if not LOAD:
    torch.save(policy.state_dict(), file_name)

if LOAD:
    policy.load_state_dict((torch.load(file_name)))

In [ ]:
with media.VideoWriter(
    "behavior cloning eval.mp4", shape=(480, 640), fps=5) as w:
    for image in image_list:
        w.add_image(image)

In [ ]:
policy.state_dict()